# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import json
import azureml.core
from azureml.core import Workspace, Experiment, Model
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
from azureml.train.automl import AutoMLConfig
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, Webservice

print("SDK version:", azureml.core.VERSION)

SDK version: 1.36.0


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

quick-starts-ws-169400
aml-quickstarts-169400
southcentralus
3e42d11f-d64d-4173-af9b-12ecaa1030b3


In [3]:
# choose a name for experiment
experiment_name = 'automl-heart-experiment'

experiment = Experiment(ws, experiment_name)

run = experiment.start_logging()

In [4]:
amlcompute_cluster_name = "heart-compute"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)

InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded..................................................................................................................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


In [5]:
key = "heart-failure"

dataset = ws.datasets[key]

df = dataset.to_pandas_dataframe()
df.describe()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
count,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.00000,299.000000,299.000000,299.00000,299.000000,299.00000
mean,60.833893,0.431438,581.839465,0.418060,38.083612,0.351171,263358.029264,1.39388,136.625418,0.648829,0.32107,130.260870,0.32107
std,11.894809,0.496107,970.287881,0.494067,11.834841,0.478136,97804.236869,1.03451,4.412477,0.478136,0.46767,77.614208,0.46767
min,40.000000,0.000000,23.000000,0.000000,14.000000,0.000000,25100.000000,0.50000,113.000000,0.000000,0.00000,4.000000,0.00000
25%,51.000000,0.000000,116.500000,0.000000,30.000000,0.000000,212500.000000,0.90000,134.000000,0.000000,0.00000,73.000000,0.00000
50%,60.000000,0.000000,250.000000,0.000000,38.000000,0.000000,262000.000000,1.10000,137.000000,1.000000,0.00000,115.000000,0.00000
75%,70.000000,1.000000,582.000000,1.000000,45.000000,1.000000,303500.000000,1.40000,140.000000,1.000000,1.00000,203.000000,1.00000
max,95.000000,1.000000,7861.000000,1.000000,80.000000,1.000000,850000.000000,9.40000,148.000000,1.000000,1.00000,285.000000,1.00000


In [6]:
dataset.take(5).to_pandas_dataframe()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [7]:


automl_settings = {
    "experiment_timeout_minutes": 15,
    "iterations": 40,
    "max_concurrent_iterations": 4,
    "n_cross_validations": 3,
    "primary_metric" : 'accuracy'
}
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="DEATH_EVENT",
                             enable_early_stopping= True,
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )


In [8]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output=True)

Submitting remote run.
No run_configuration provided, running on heart-compute with default configuration
Running on remote compute: heart-compute


Experiment,Id,Type,Status,Details Page,Docs Page
automl-heart-experiment,AutoML_3bfa6805-f343-4c31-8548-f97257b3c500,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [9]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [10]:
remote_run.wait_for_completion(show_output=True)

Experiment,Id,Type,Status,Details Page,Docs Page
automl-heart-experiment,AutoML_3bfa6805-f343-4c31-8548-f97257b3c500,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation




****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.
              Learn more abo

{'runId': 'AutoML_3bfa6805-f343-4c31-8548-f97257b3c500',
 'target': 'heart-compute',
 'status': 'Completed',
 'startTimeUtc': '2021-12-27T14:51:30.107816Z',
 'endTimeUtc': '2021-12-27T15:08:36.812883Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'No scores improved over last 20 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}],
 'properties': {'num_iterations': '40',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '3',
  'target': 'heart-compute',
  'AMLSettingsJsonString': '{"path":null,"name":"automl-heart-experiment","subscription_id":"3e42d11f-d64d-4173-af9b-12ecaa1030b3","resource_group":"aml-quickstarts-169400","workspace_name":"quick-starts-ws-169400","region":"southcentralus","compute_target":"hear

In [11]:
best_run, fitted_model = remote_run.get_output()

In [12]:
best_metrics = best_run.get_metrics()

print("Best Run Id : {}".format(best_run.id), 
      "Accuracy : {}".format(best_metrics['accuracy']), 
      "Best metrics : {}".format(best_metrics), 
      "Best model : {}".format(fitted_model), sep = '\n')

Best Run Id : AutoML_3bfa6805-f343-4c31-8548-f97257b3c500_36
Accuracy : 0.8661952861952863
Best metrics : {'f1_score_weighted': 0.8608884425043768, 'precision_score_micro': 0.8661952861952863, 'matthews_correlation': 0.6924546051394648, 'average_precision_score_weighted': 0.9001450148681682, 'recall_score_macro': 0.8285046129873717, 'AUC_weighted': 0.8926262715917889, 'f1_score_macro': 0.8377222202469939, 'f1_score_micro': 0.8661952861952863, 'recall_score_micro': 0.8661952861952863, 'log_loss': 0.43032743447102967, 'weighted_accuracy': 0.8960318808504794, 'AUC_macro': 0.8926262715917889, 'precision_score_macro': 0.8663370922962957, 'accuracy': 0.8661952861952863, 'precision_score_weighted': 0.8716420104178454, 'average_precision_score_micro': 0.9033527361098, 'balanced_accuracy': 0.8285046129873717, 'recall_score_weighted': 0.8661952861952863, 'average_precision_score_macro': 0.8743104897314934, 'norm_macro_recall': 0.6570092259747432, 'AUC_micro': 0.9023028942624903, 'accuracy_table'

In [13]:
print(fitted_model._final_estimator)

PreFittedSoftVotingClassifier(
    estimators=[('5', Pipeline(
        memory=None,
        steps=[('maxabsscaler', MaxAbsScaler(
            copy=True
        )), ('lightgbmclassifier', LightGBMClassifier(
            boosting_type='gbdt',
            colsample_bytree=0.6933333333333332,
            learning_rate=0.09473736842105263,
            max_bin=110,
            max_depth=8,
            min_child_weight=6,
            min_data_in_leaf=0.003457931034482759,
            min_split_gain=1,
            n_estimators=25,
            num_leaves=227,
            reg_alpha=0.9473684210526315,
            reg_lambda=0.42105263157894735,
            subsample=0.49526315789473685,
            random_state=None,
            n_jobs=1,
            problem_info=ProblemInfo(
                gpu_training_param_dict={'processing_unit_type': 'cpu'}
            )
        ))],
        verbose=False
    )), ('31', Pipeline(
        memory=None,
        steps=[('standardscalerwrapper', StandardScalerW

In [14]:
for model in Model.list(ws):
    print(model.name, 'version:', model.version)

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [15]:
#Save the best model for the deployement
import os
os.makedirs('./aml-model', exist_ok=True)

best_run.download_file('/outputs/model.pkl',os.path.join('./aml-model','heart-failure-best-model-automl.pkl'))

for f in best_run.get_file_names():
    if f.startswith('outputs'):
        output_file_path = os.path.join('./aml-model', f.split('/')[-1])
        print(f'Downloading from {f} to {output_file_path} ...')
        best_run.download_file(name=f, output_file_path=output_file_path)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [17]:
model=best_run.register_model(
            model_name = 'heart-failure-best-model-automl', 
            model_path = './outputs/model.pkl'
)

TODO: In the cell below, print the logs of the web service and delete the service

In [18]:
best_run.download_file('outputs/conda_env_v_1_0_0.yml', 'conda_env.yml')
environment = Environment.from_conda_specification(name = 'heart-failure-env', file_path = 'conda_env.yml')

In [19]:
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'score.py')

In [20]:
inference_config = InferenceConfig(entry_script= 'score.py', environment=environment)

In [21]:
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1, auth_enabled=True, enable_app_insights=True)

In [22]:
webservice = Model.deploy(workspace=ws, 
                        name='heart-failure-ws', 
                        models=[model], 
                        inference_config=inference_config,
                        deployment_config=aci_config,
                        overwrite=True)

In [23]:
webservice

AciWebservice(workspace=Workspace.create(name='quick-starts-ws-169400', subscription_id='3e42d11f-d64d-4173-af9b-12ecaa1030b3', resource_group='aml-quickstarts-169400'), name=heart-failure-ws, image_id=None, compute_type=None, state=ACI, scoring_uri=Unhealthy, tags=None, properties={}, created_by={})

In [24]:
# wait for deployment to finish and display the scoring uri and swagger uri
webservice.wait_for_deployment(show_output=True)

print("Status : {}".format(webservice.state), 
      "scoring_uri : {}".format(webservice.scoring_uri), 
      "swagger_uri : {}".format(webservice.swagger_uri), sep = '\n')

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-12-27 15:32:00+00:00 Creating Container Registry if not exists..
2021-12-27 15:42:01+00:00 Registering the environment..
2021-12-27 15:42:02+00:00 Building image..
2021-12-27 15:53:16+00:00 Generating deployment configuration.
2021-12-27 15:53:17+00:00 Submitting deployment to compute..
2021-12-27 15:53:21+00:00 Checking the status of deployment heart-failure-ws..
2021-12-27 15:55:51+00:00 Checking the status of inference endpoint heart-failure-ws.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Status : Healthy
scoring_uri : http://e433f074-f50b-4751-ba6a-35ba6d05febf.southcentralus.azurecontainer.io/score
swagger_uri : http://e433f074-f50b-4751-ba6a-35ba6d05febf.southcentralus.azurecontainer.io/swagger.json


In [25]:
from pprint import pprint

# select samples from the dataframe
test_data = df.sample(5)
test_label = test_data.pop('DEATH_EVENT')

# convert the records to a json data file
data_json = test_data.to_dict(orient='records')

data = json.dumps({'data': data_json})
pprint(data)

('{"data": [{"age": 66.0, "anaemia": 1, "creatinine_phosphokinase": 68, '
 '"diabetes": 1, "ejection_fraction": 38, "high_blood_pressure": 1, '
 '"platelets": 162000.0, "serum_creatinine": 1.0, "serum_sodium": 136, "sex": '
 '0, "smoking": 0, "time": 95}, {"age": 58.0, "anaemia": 1, '
 '"creatinine_phosphokinase": 60, "diabetes": 0, "ejection_fraction": 38, '
 '"high_blood_pressure": 0, "platelets": 153000.0, "serum_creatinine": 5.8, '
 '"serum_sodium": 134, "sex": 1, "smoking": 0, "time": 26}, {"age": 56.0, '
 '"anaemia": 1, "creatinine_phosphokinase": 135, "diabetes": 1, '
 '"ejection_fraction": 38, "high_blood_pressure": 0, "platelets": 133000.0, '
 '"serum_creatinine": 1.7, "serum_sodium": 140, "sex": 1, "smoking": 0, '
 '"time": 244}, {"age": 50.0, "anaemia": 1, "creatinine_phosphokinase": 298, '
 '"diabetes": 0, "ejection_fraction": 35, "high_blood_pressure": 0, '
 '"platelets": 362000.0, "serum_creatinine": 0.9, "serum_sodium": 140, "sex": '
 '1, "smoking": 1, "time": 240}, {"ag

In [26]:
# get inference
output = webservice.run(data)
print(output)

{"result": [0, 1, 0, 0, 0]}


In [27]:
webservice.get_logs()

'2021-12-27T15:55:40,467535500+00:00 - gunicorn/run \nDynamic Python package installation is disabled.\nStarting HTTP server\n2021-12-27T15:55:40,466551400+00:00 - iot-server/run \n2021-12-27T15:55:40,479013700+00:00 - rsyslog/run \n2021-12-27T15:55:40,516845800+00:00 - nginx/run \nrsyslogd: /azureml-envs/azureml_bec455c772c2d018a0dc9c15319d853a/lib/libuuid.so.1: no version information available (required by rsyslogd)\nEdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...\n2021-12-27T15:55:40,839288800+00:00 - iot-server/finish 1 0\n2021-12-27T15:55:40,841773400+00:00 - Exit code 1 is normal. Not restarting iot-server.\nStarting gunicorn 20.1.0\nListening at: http://127.0.0.1:31311 (72)\nUsing worker: sync\nworker timeout is set to 300\nBooting worker with pid: 100\nSPARK_HOME not set. Skipping PySpark Initialization.\nGenerating new fontManager, this may take some time...\nInitializing logger\n2021-12-27 15:55:44,053 | root | INFO | Starting up app insights client

In [28]:
# select samples from the dataframe
test_data = df.sample(10)
test_label = test_data.pop('DEATH_EVENT')

# convert the records to a json data file
data_json = test_data.to_dict(orient='records')

data = json.dumps({'data': data_json})

# get inference
output = webservice.run(data)
print(output)

{"result": [1, 1, 1, 0, 1, 0, 0, 0, 0, 0]}


In [29]:
webservice.get_logs()

'2021-12-27T15:55:40,467535500+00:00 - gunicorn/run \nDynamic Python package installation is disabled.\nStarting HTTP server\n2021-12-27T15:55:40,466551400+00:00 - iot-server/run \n2021-12-27T15:55:40,479013700+00:00 - rsyslog/run \n2021-12-27T15:55:40,516845800+00:00 - nginx/run \nrsyslogd: /azureml-envs/azureml_bec455c772c2d018a0dc9c15319d853a/lib/libuuid.so.1: no version information available (required by rsyslogd)\nEdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...\n2021-12-27T15:55:40,839288800+00:00 - iot-server/finish 1 0\n2021-12-27T15:55:40,841773400+00:00 - Exit code 1 is normal. Not restarting iot-server.\nStarting gunicorn 20.1.0\nListening at: http://127.0.0.1:31311 (72)\nUsing worker: sync\nworker timeout is set to 300\nBooting worker with pid: 100\nSPARK_HOME not set. Skipping PySpark Initialization.\nGenerating new fontManager, this may take some time...\nInitializing logger\n2021-12-27 15:55:44,053 | root | INFO | Starting up app insights client

In [30]:
webservice.delete()
compute_target.delete()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
